In [6]:
import numpy as np
from PIL import Image
import torch
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity as lpips
from torchvision import transforms
from SIT import *


In [7]:
epsilon = 16/255
alpha = 0.6/255
epochs = 10
decay = 1

lpips_loss = lpips(net_type='squeeze', normalize=True)

# Utils

In [8]:
def add_noise(x, noise=None):
        if noise != None:
               return torch.clip(x + noise, 0, 1)
        return torch.clip(x + torch.zeros_like(x).uniform_(-16/255,16/255), 0, 1)

def tensor_to_image(image):
      img = image.permute(1, 2, 0).cpu().detach().numpy() * 255
      img = img.astype(np.uint8)
      img = Image.fromarray(img)
      return img

def give_sit_delta(data):
      data_copy = blocktransform(data)
      delta = data_copy - data
      return delta

def init_delta(data, method=None):
      if method == None:
            delta = torch.zeros_like(data)
            delta.uniform_(-epsilon, epsilon)
            delta.requires_grad = True
            return delta
      elif method == "SIT":
        delta = give_sit_delta(data)

      return delta

def get_loss(image1, image2):
       res = lpips_loss(image1, image2)
       return res

def update_delta(delta, grad):
      delta = torch.clamp(delta + alpha * grad.sign(), -epsilon, epsilon)
      return delta
       
def change_view(data):
      _, x, y = data.shape
      data = data.view(1,_,x,y)
      return data

image_path = "dog.jpg"

transform = transforms.Compose([
  transforms.ToTensor()  # Convert to tensor with pixel values between 0 and 1
])

# Lpips Optimization

In [4]:
image = Image.open(image_path).convert('RGB')

transformed_image = transform(image)
transformed_image.requires_grad = True

transformed_image = change_view(transformed_image)

# delta = init_delta(transformed_image, method="SIT")
delta = init_delta(transformed_image)
noised_image = add_noise(transformed_image, delta)

momentum = 0.
for _ in range(10):
        loss = get_loss(transformed_image, noised_image)
        grad = torch.autograd.grad(loss, delta)[0]
        momentum = momentum * decay + grad / (grad.abs().mean(dim=(1,2,3), keepdim=True))
        delta = update_delta(delta, momentum)
        noised_image = add_noise(noised_image, delta)
        new_img = tensor_to_image(noised_image[0])
        new_img.save(f'LPIPS_Images/Lpips_image_{_}.jpg')
        print(loss)

tensor(0.4765, grad_fn=<SqueezeBackward0>)
tensor(0.6789, grad_fn=<SqueezeBackward0>)
tensor(0.7920, grad_fn=<SqueezeBackward0>)
tensor(0.8667, grad_fn=<SqueezeBackward0>)
tensor(0.9236, grad_fn=<SqueezeBackward0>)
tensor(0.9685, grad_fn=<SqueezeBackward0>)
tensor(1.0019, grad_fn=<SqueezeBackward0>)
tensor(1.0275, grad_fn=<SqueezeBackward0>)
tensor(1.0487, grad_fn=<SqueezeBackward0>)
tensor(1.0671, grad_fn=<SqueezeBackward0>)


# SIT After LPIPS Optimization

In [5]:
lpips_image = Image.open("LPIPS_Images/Lpips_image_1.jpg").convert("RGB")

lpips_tensor = transform(lpips_image)

lpips_tensor = change_view(lpips_tensor)
lpips_tensor = add_noise(lpips_tensor ,give_sit_delta(lpips_tensor))

new_score = get_loss(transformed_image, lpips_tensor)
print(new_score)

after_sit_image = tensor_to_image(lpips_tensor[0])
after_sit_image.show()

tensor(0.8711, grad_fn=<SqueezeBackward0>)


# Creating Adverserial Example